## Steps:
#### 1.Last expiry date in jan:
FInal tranact expiry

#### 2.first transaction after transaction that caused expiry date in jan:
* Check expiry later than jan 31 and order ascendingly.
* remove 'is_Cancel'=1 transactions.
* choose transactions later than december 31.
* choose first row for each user.
* choose transaction_date column. 

Subtract 1 from 2. if its <31 then no churn. otherwise churn.
Users who are missing in 2,never done any transaction again. meaning they churned.

In [1]:
import pandas as pd

transaction=pd.read_csv('transactions.csv')

In [2]:
import numpy as np

transaction_last_expiry_msno_final=np.load('transaction_last_expiry_msno_final.npy', allow_pickle=True)

In [3]:
Train_msno_and_label_order=np.load('Train_msno_and_label_order.npy', allow_pickle=True)

In [7]:
#select msno only in train
transaction_train_msno=transaction[transaction['msno'].isin(Train_msno_and_label_order[:,0])]

In [8]:
#Users with expiry date after jan. 
transaction_train_msno_churn=transaction_train_msno[transaction_train_msno['membership_expire_date']>20170131]

In [9]:
transaction_train_msno_churn=transaction_train_msno_churn.sort_values(by=['msno','transaction_date'],ascending=[True,True])

In [10]:
transaction_train_msno_churn

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
58566,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,1,20170115,20170215,0
1914756,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,1,20170215,20170315,0
18666063,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,39,30,149,149,1,20161231,20170219,0
16166417,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,39,30,149,149,1,20170131,20170319,0
8526712,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,41,30,149,149,1,20170126,20170226,0
...,...,...,...,...,...,...,...,...,...
1862627,zzz1Dc3P9s53HAowRTrm3fNsWju5yeN4YBfNDq7Z99Q=,39,30,149,149,1,20161231,20170224,0
2006696,zzz1Dc3P9s53HAowRTrm3fNsWju5yeN4YBfNDq7Z99Q=,39,30,149,149,1,20170131,20170324,0
15515807,zzz4xr5LLzoxUqD6LfKiieCt8Or4dUWFQ0RA5gmkx8I=,41,30,149,149,1,20170104,20170205,0
20411713,zzzN9thH22os1dRS0VHReY/8FTfGHOi86//d+wGGFsQ=,41,30,99,99,1,20170104,20170204,0


In [11]:
#remove is_cancel=1 transactions
transaction_train_msno_churn=transaction_train_msno_churn[transaction_train_msno_churn['is_cancel']==0]

In [12]:
#transaction later than dec 31
transaction_train_msno_churn=transaction_train_msno_churn[transaction_train_msno_churn['transaction_date']>20161231]

In [13]:
#https://stackoverflow.com/a/15705958
#there are also other ways to do this. eg:drop duplicates will work.
index=transaction_train_msno_churn.groupby(['msno'])['transaction_date'].transform(min) == transaction_train_msno_churn['transaction_date']

In [14]:
transaction_train_msno_churn=transaction_train_msno_churn[index]

In [15]:
transaction_train_msno_churn #contains users who have transacted after expiry in jan and their first transaction after jan

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
58566,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,1,20170115,20170215,0
16166417,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,39,30,149,149,1,20170131,20170319,0
8526712,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,41,30,149,149,1,20170126,20170226,0
6934298,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,41,30,149,149,1,20170115,20170215,0
12261060,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,39,30,149,149,1,20170131,20170323,0
...,...,...,...,...,...,...,...,...,...
17324717,zzxqpgPX1epGyR3SVRKb3XitWdt+JqDURxV5ea1D7Cg=,38,30,149,149,0,20170127,20170301,0
6971843,zzy7iqSpfcRq7R4hmKKuhI+CJRs79a6pteqEggpiNO0=,36,30,180,180,0,20170131,20170302,0
2006696,zzz1Dc3P9s53HAowRTrm3fNsWju5yeN4YBfNDq7Z99Q=,39,30,149,149,1,20170131,20170324,0
15515807,zzz4xr5LLzoxUqD6LfKiieCt8Or4dUWFQ0RA5gmkx8I=,41,30,149,149,1,20170104,20170205,0


In [16]:
#load final transact 
import pandas as pd

final_transact=pd.read_csv('final_transact_train.csv')

In [17]:
final_transact

,Unnamed: 0,msno,Label,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,0,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,0,38.0,410.0,1788.0,1788.0,0.0,20151121.0,20170104.0,0.0
1,1,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,1,41.0,30.0,99.0,99.0,1.0,20161215.0,20170115.0,0.0
2,2,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,1,39.0,30.0,149.0,149.0,1.0,20161130.0,20170119.0,0.0
3,3,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,1,41.0,30.0,149.0,149.0,1.0,20161226.0,20170126.0,0.0
4,4,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,1,41.0,30.0,149.0,149.0,1.0,20161215.0,20170115.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
956322,956322,zzy7iqSpfcRq7R4hmKKuhI+CJRs79a6pteqEggpiNO0=,1,36.0,30.0,180.0,180.0,0.0,20161230.0,20170129.0,0.0
956323,956323,zzyHq6TK2+cBkeGFUHvh12Z7UxFZiSM7dOOSllSBPDw=,0,41.0,30.0,149.0,149.0,1.0,20161214.0,20170114.0,0.0
956324,956324,zzz1Dc3P9s53HAowRTrm3fNsWju5yeN4YBfNDq7Z99Q=,1,39.0,30.0,149.0,149.0,1.0,20161130.0,20170124.0,0.0
956325,956325,zzz4xr5LLzoxUqD6LfKiieCt8Or4dUWFQ0RA5gmkx8I=,1,41.0,30.0,149.0,149.0,1.0,20161204.0,20170105.0,0.0


In [18]:
final_transact=final_transact[['msno','membership_expire_date']]

In [19]:
final_transact.columns=['msno', 'jan_final_expiry']

In [20]:
final_transact.head()

,msno,jan_final_expiry
0,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,20170104.0
1,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,20170115.0
2,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,20170119.0
3,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,20170126.0
4,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,20170115.0


In [21]:
final_transact=final_transact[final_transact['msno'].isin(transaction_train_msno_churn['msno'].values)]

In [22]:
mer=pd.merge(transaction_train_msno_churn, final_transact, on='msno')

In [23]:
mer=mer[['msno','transaction_date','jan_final_expiry']]

In [24]:
mer

,msno,transaction_date,jan_final_expiry
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,20170115,20170115.0
1,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,20170131,20170119.0
2,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,20170126,20170126.0
3,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,20170115,20170115.0
4,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,20170131,20170123.0
...,...,...,...
885640,zzxqpgPX1epGyR3SVRKb3XitWdt+JqDURxV5ea1D7Cg=,20170127,20170130.0
885641,zzy7iqSpfcRq7R4hmKKuhI+CJRs79a6pteqEggpiNO0=,20170131,20170129.0
885642,zzz1Dc3P9s53HAowRTrm3fNsWju5yeN4YBfNDq7Z99Q=,20170131,20170124.0
885643,zzz4xr5LLzoxUqD6LfKiieCt8Or4dUWFQ0RA5gmkx8I=,20170104,20170105.0


In [25]:
def fun(x):
    '''returns true if no of days between x[1] and x[2] is less than or equal to 1 month'''
    return (((x[1]//10000*365)+(x[1]%10000//100*30.4167)+(x[1]%100))-
            ((x[2]//10000*365)+(x[2]%10000//100*30.4167)+(x[2]%100)))<31
    

In [26]:
c=mer.apply(fun, axis=1)

In [27]:
c[0:5]

0    True
1    True
2    True
3    True
4    True
dtype: bool

In [28]:
not_churn=mer[c]

In [29]:
not_churn 

,msno,transaction_date,jan_final_expiry
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,20170115,20170115.0
1,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,20170131,20170119.0
2,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,20170126,20170126.0
3,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,20170115,20170115.0
4,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,20170131,20170123.0
...,...,...,...
885640,zzxqpgPX1epGyR3SVRKb3XitWdt+JqDURxV5ea1D7Cg=,20170127,20170130.0
885641,zzy7iqSpfcRq7R4hmKKuhI+CJRs79a6pteqEggpiNO0=,20170131,20170129.0
885642,zzz1Dc3P9s53HAowRTrm3fNsWju5yeN4YBfNDq7Z99Q=,20170131,20170124.0
885643,zzz4xr5LLzoxUqD6LfKiieCt8Or4dUWFQ0RA5gmkx8I=,20170104,20170105.0


In [30]:
not_churn=not_churn.drop_duplicates( subset=['msno'], keep="first", inplace=False) #remove duplicates

In [31]:
not_churn

,msno,transaction_date,jan_final_expiry
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,20170115,20170115.0
1,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,20170131,20170119.0
2,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,20170126,20170126.0
3,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,20170115,20170115.0
4,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,20170131,20170123.0
...,...,...,...
885640,zzxqpgPX1epGyR3SVRKb3XitWdt+JqDURxV5ea1D7Cg=,20170127,20170130.0
885641,zzy7iqSpfcRq7R4hmKKuhI+CJRs79a6pteqEggpiNO0=,20170131,20170129.0
885642,zzz1Dc3P9s53HAowRTrm3fNsWju5yeN4YBfNDq7Z99Q=,20170131,20170124.0
885643,zzz4xr5LLzoxUqD6LfKiieCt8Or4dUWFQ0RA5gmkx8I=,20170104,20170105.0


In [32]:
train_not_churn_final_msno=np.array(not_churn['msno'].to_numpy())

In [33]:
train_not_churn_final_msno.shape

(877738,)

In [34]:
np.save('train_not_churn_final_msno_new',train_not_churn_final_msno) #users who have not churned

In [35]:
956327-877738

78589

In [36]:
78589/956327*100 #churn datapoints 8.2% but many datapoints will be removed from this set.

8.217795795789517

### Join churn and not churn datapoints:

In [41]:
#all train points
transaction_last_expiry_msno_final=np.load('transaction_last_expiry_msno_final.npy', allow_pickle=True)

In [37]:
not_ch=[1]*877738 #label=1

In [38]:
not_ch=np.vstack((train_not_churn_final_msno,not_ch))

In [39]:
not_ch=not_ch.T 

In [40]:
not_ch[0:2]

array([['+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=', 1],
       ['+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=', 1]], dtype=object)

In [43]:
transaction_last_expiry_msno_final[0]

array(['+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=', 38, 410, 1788,
       1788, 0, 20151121, 20170104, 0], dtype=object)

In [42]:
churn=set(transaction_last_expiry_msno_final[:,0])-set(train_not_churn_final_msno)

In [43]:
churn=np.array(list(churn))

In [44]:
churn.shape

(78589,)

In [45]:
ch=[0]*78589 #label=0

In [46]:
ch=np.vstack((churn,ch)).T

In [47]:
ch[0] #churn datapoints

array(['8u1LB9fm/i9oZ2pAyw2J44Kx4shNqHoacpEAmWs9eIc=', '0'], dtype='<U44')

In [48]:
Train=np.vstack((ch,not_ch)) #0=churn, 1=not churn

In [49]:
Train.shape

(956327, 2)

In [50]:
Train[0:5]

array([['8u1LB9fm/i9oZ2pAyw2J44Kx4shNqHoacpEAmWs9eIc=', '0'],
       ['AuejxMNrrsoV4NIXqaCeFsfYMWDips6Z08gOuJQ/K00=', '0'],
       ['1y+rHMAEJGJXMwKmcx9h7AswkLwf6yXGIZWXEUMo7xw=', '0'],
       ['sSEMPqbKd4mLVXDUXjhHZA4Jn/ozZBF7z7E0YvqSRTo=', '0'],
       ['KNkgvudj3KeuTRbydWNUUiSFNsmnIIJ0M6VORM0w6wU=', '0']],
      dtype=object)

In [51]:
#sort based on msno
#https://stackoverflow.com/a/2828121
Train=Train[Train[:,0].argsort()]

In [52]:
Train[0:5]

array([['+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=', '0'],
       ['+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=', 1],
       ['+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=', 1],
       ['+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=', 1],
       ['++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=', 1]], dtype=object)

In [53]:
np.save('Train_msno_and_label_order_new',Train)